<h1 align="center">Summary of the tasks: How to run the models of the Transformers library task by task</h1>

Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro

# Resumo das tarefas

## Instalamos a Biblioteca Transformer

In [1]:
# Transformers installation
%pip install transformers datasets

# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

You should consider upgrading via the '/home/eddygiusepe/24_Transformers_NLP/NLP_Transformers/venv_transformers/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


Esta página mostra os casos de uso mais frequentes ao usar a biblioteca. Os modelos disponíveis permitem diversas configurações e uma grande versatilidade nos casos de uso. As mais simples são apresentadas aqui, mostrando o uso para tarefas como `classificação de imagens` (Imagem classification), `resposta a perguntas` (Question Answering), `classificação de sequências`, `reconhecimento de entidades nomeadas` (NER) e outras.

Esses exemplos utilizam `auto-models`, que são classes que instanciarão um modelo de acordo com um determinado `checkpoint`, selecionando automaticamente a arquitetura de modelo correta. Verifique a documentação do [AutoModel](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModel) para obter mais informações. Sinta-se à vontade para modificar o código para ser mais específico e adaptá-lo ao seu caso de uso específico.

Para que um modelo tenha um bom desempenho em uma tarefa, ele deve ser carregado a partir de um checkpoint correspondente a essa tarefa. Esses pontos de verificação geralmente são `pré-treinados` em um grande corpus de dados e ajustados (`fine-tuned`) para uma tarefa específica. Isso significa o seguinte:

* Nem todos os modelos foram fine-tuned em todas as tarefas. Se você deseja fine-tune um modelo em uma tarefa específica, pode aproveitar um dos scripts `run_$TASK.py` no diretório de [exemplos](https://github.com/huggingface/transformers/tree/main/examples).


* Modelos ajustados (fine-tuned) foram ajustados em um conjunto de dados específico. Este conjunto de dados pode ou não se sobrepor ao seu caso de uso e domínio. Conforme mencionado anteriormente, você pode aproveitar os scripts de [exemplos](https://github.com/huggingface/transformers/tree/main/examples) para ajustar seu modelo ou pode criar seu próprio script de treinamento.


Para fazer uma inferência sobre uma tarefa, diversos mecanismos são disponibilizados pela biblioteca:

* `Pipelines`: abstrações muito fáceis de usar, que requerem apenas duas linhas de código.

* `Uso direto do modelo`: menos abstrações, mas mais flexibilidade e poder por meio de acesso direto a um `tokenizador` (`PyTorch/TensorFlow`) e capacidade total de inferência.


Ambas as abordagens são apresentadas aqui.

Todas as tarefas apresentadas aqui utilizam `checkpoint` pré-treinados que foram ajustados em tarefas específicas. Carregar um ponto de verificação que não foi ajustado em uma tarefa específica carregaria apenas as camadas do `Transformer` de base e não o `head adicional` que é usado para a tarefa, inicializando os pesos desse head aleatoriamente.

Isso produziria uma saída aleatória.

# Sequence Classification

A `classificação de sequências` é a tarefa de classificar as sequências de acordo com um determinado número de classes. <font color="yellow">Um exemplo de classificação de sequência</font> é o conjunto de dados `GLUE`, que é totalmente baseado nessa tarefa. Se você deseja ajustar um modelo em uma tarefa de classificação de sequência GLUE, pode aproveitar os scripts [run_glue.py](https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification/run_glue.py), run_tf_glue.py, run_tf_text_classification.py ou [run_xnli.py](https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification/run_xnli.py).

Aqui está um exemplo de uso de pipelines para fazer `análise de sentimento`: identificar se uma sequência é `positiva` ou `negativa`. Ele aproveita um modelo fine-tuned em `sst2`, que é uma tarefa `GLUE`.

Isso retorna uma Label (`"POSITIVO"` ou `"NEGATIVO"`) ao lado de uma pontuação, como segue:

In [3]:
from transformers import pipeline


classifier = pipeline("sentiment-analysis")

result = classifier("I love studying Machine Learning, that's why I became a Data Scientist.")[0]
print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


label: POSITIVE, with score: 0.9982


In [4]:
result = classifier("I hate people who don't fight for their dreams.")[0]
print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

label: NEGATIVE, with score: 0.9642


Aqui está um exemplo de como fazer uma classificação de sequência usando um modelo para determinar se duas sequências são paráfrases uma da outra. O processo é o seguinte:

<font color="orange">1.</font> Instancie um tokenizer e um modelo a partir do nome do `checkpoint`. O modelo é identificado como um `modelo BERT` e o carrega com os pesos armazenados no checkpoint.

<font color="orange">2.</font> Construa uma sequência a partir das duas sentenças, com os separadores corretos e específicos do modelo, `ids` de tipo de token e `attention masks` (que serão criadas automaticamente pelo tokenizador).

<font color="orange">3.</font> Passe essa sequência pelo modelo para que ela seja classificada em uma das duas classes disponíveis: $0$$ (`não é uma paráfrase`) e $1$ (é uma paráfrase).

<font color="orange">4.</font> Calcule o `softmax` do resultado para obter probabilidades sobre as classes.

<font color="orange">5.</font> Imprima os resultados.

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased-finetuned-mrpc")

classes = ["not paraphrase", "is paraphrase"]

sequence_0 = "The company HuggingFace is based in New York City"
sequence_1 = "Apples are especially bad for your health"
sequence_2 = "HuggingFace's headquarters are situated in Manhattan"

# The tokenizer will automatically add any model specific separators (i.e. <CLS> and <SEP>) and tokens to
# the sequence, as well as compute the attention masks.
paraphrase = tokenizer(sequence_0, sequence_2, return_tensors="pt")
not_paraphrase = tokenizer(sequence_0, sequence_1, return_tensors="pt")

paraphrase_classification_logits = model(**paraphrase).logits
not_paraphrase_classification_logits = model(**not_paraphrase).logits

paraphrase_results = torch.softmax(paraphrase_classification_logits, dim=1).tolist()[0]
not_paraphrase_results = torch.softmax(not_paraphrase_classification_logits, dim=1).tolist()[0]

print("A seguir comparamos a sequence_0 e a sequence_2, as quais SÃO PARÂFRASES! ")
# Should be paraphrase
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(paraphrase_results[i] * 100))}%")

A seguir comparamos a sequence_0 e a sequence_2, as quais SÃO PARÂFRASES! 
not paraphrase: 10%
is paraphrase: 90%


In [12]:
# Should not be paraphrase
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(not_paraphrase_results[i] * 100))}%")


not paraphrase: 94%
is paraphrase: 6%


In [23]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased-finetuned-mrpc")

classes = ["not paraphrase", "is paraphrase"]

sequence_0 = "The company HuggingFace is based in New York City"
sequence_1 = "Apples are especially bad for your health"
sequence_2 = "HuggingFace's headquarters are situated in Manhattan"

# The tokenizer will automatically add any model specific separators (i.e. <CLS> and <SEP>) and tokens to
# the sequence, as well as compute the attention masks.
paraphrase = tokenizer(sequence_0, sequence_2, return_tensors="tf")
not_paraphrase = tokenizer(sequence_0, sequence_1, return_tensors="tf")

paraphrase_classification_logits = model(paraphrase).logits
not_paraphrase_classification_logits = model(not_paraphrase).logits

paraphrase_results = tf.nn.softmax(paraphrase_classification_logits, axis=1).numpy()[0]
not_paraphrase_results = tf.nn.softmax(not_paraphrase_classification_logits, axis=1).numpy()[0]

# Should be paraphrase
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(paraphrase_results[i] * 100))}%")

Some layers from the model checkpoint at bert-base-cased-finetuned-mrpc were not used when initializing TFBertForSequenceClassification: ['dropout_183']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at bert-base-cased-finetuned-mrpc.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


not paraphrase: 10%
is paraphrase: 90%


In [24]:
# Should not be paraphrase
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(not_paraphrase_results[i] * 100))}%")

not paraphrase: 94%
is paraphrase: 6%


# Extractive Question Answering

Extrative Question Answering é a tarefa de `extrair uma resposta` de um texto a partir de uma pergunta. Um exemplo de um conjunto de dados de resposta a perguntas é o conjunto de dados `SQuAD`, que é totalmente baseado nessa tarefa. Se desejar ajustar (`fine-tune`) um modelo em uma tarefa SQuAD, você pode aproveitar os scripts [run_qa.py](https://github.com/huggingface/transformers/blob/main/examples/pytorch/question-answering/run_qa.py) e `run_tf_squad.py`.

Aqui está um exemplo de uso de `pipelines` para responder a perguntas: extrair uma resposta de um texto dado a uma pergunta. Ele aproveita um modelo `fine-tuned` no SQUAD.

In [14]:
from transformers import pipeline

question_answerer = pipeline("question-answering")


context = r"""
Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a
question answering dataset is the SQuAD dataset, which is entirely based on that task. If you would like to fine-tune
a model on a SQuAD task, you may leverage the examples/pytorch/question-answering/run_squad.py script.
"""

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


Isso retorna uma `resposta extraída do texto`, uma pontuação de confiança, juntamente com os valores `"start"` e `"end"`, que são as posições da resposta extraída no texto.

In [15]:
result = question_answerer(question="What is extractive question answering?", context=context)

print(
    f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}"
)


Answer: 'the task of extracting an answer from a text given a question', score: 0.6177, start: 34, end: 95


In [16]:
result = question_answerer(question="What is a good example of a question answering dataset?", context=context)
print(
    f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}"
)

Answer: 'SQuAD dataset', score: 0.5152, start: 147, end: 160


Aqui está um exemplo de resposta a perguntas (`Question Answering`) usando um modelo e um tokenizador. O processo é o seguinte:


<font color="orange">1.</font> Instancie um tokenizador e um modelo a partir do nome do checkpoint. O modelo é identificado como um `modelo BERT` e o carrega com os pesos armazenados no ponto de verificação.

<font color="orange">2.</font> Defina um texto e algumas perguntas.

<font color="orange">3.</font> Repita as perguntas e crie uma sequência a partir do texto e da pergunta atual, com os separadores corretos específicos do modelo, `IDs` de tipo de token e máscaras de atenção.

<font color="orange">4.</font> Passe esta sequência pelo modelo. Isso gera uma variedade de pontuações em todos os tokens de sequência (`pergunta e texto`), para as posições inicial e final.

<font color="orange">5.</font> Calcule o `softmax` do resultado para obter probabilidades sobre os tokens.

<font color="orange">6.</font> Busque os tokens dos valores de início (`start`) e parada (`stop`) identificados, converta esses tokens em uma string.

<font color="orange">7.</font> Imprima os resultados.

In [21]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

text = r"""
🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose
architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural
Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between
TensorFlow 2.0 and PyTorch.
"""

questions = [
    "How many pretrained models are available in 🤗 Transformers?",
    "What does 🤗 Transformers provide?",
    "🤗 Transformers provides interoperability between which frameworks?",
]

for question in questions:
    inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    # Get the most likely beginning of answer with the argmax of the score
    answer_start = torch.argmax(answer_start_scores)
    # Get the most likely end of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1

    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end])
    )

    print(f"Question: {question}")
    print(f"Answer: {answer}")

Question: How many pretrained models are available in 🤗 Transformers?
Answer: over 32 +
Question: What does 🤗 Transformers provide?
Answer: general - purpose architectures
Question: 🤗 Transformers provides interoperability between which frameworks?
Answer: tensorflow 2. 0 and pytorch


In [26]:
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = TFAutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

text = r"""
🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose
architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural
Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between
TensorFlow 2.0 and PyTorch.
"""

questions = [
    "How many pretrained models are available in 🤗 Transformers?",
    "What does 🤗 Transformers provide?",
    "🤗 Transformers provides interoperability between which frameworks?",
]

for question in questions:
    inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="tf")
    input_ids = inputs["input_ids"].numpy()[0]

    outputs = model(inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    # Get the most likely beginning of answer with the argmax of the score
    answer_start = tf.argmax(answer_start_scores, axis=1).numpy()[0]
    # Get the most likely end of answer with the argmax of the score
    answer_end = tf.argmax(answer_end_scores, axis=1).numpy()[0] + 1

    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end])
    )

    print(f"Question: {question}")
    print(f"Answer: {answer}")

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

All the layers of TFBertForQuestionAnswering were initialized from the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


Question: How many pretrained models are available in 🤗 Transformers?
Answer: over 32 +
Question: What does 🤗 Transformers provide?
Answer: general - purpose architectures
Question: 🤗 Transformers provides interoperability between which frameworks?
Answer: tensorflow 2. 0 and pytorch


# Language Modeling (Modelagem de linguagem)

A `modelagem de linguagem` é a tarefa de ajustar (fitting) um modelo a um corpus, que pode ser específico de um domínio. Todos os modelos baseados em Transformer populares são treinados usando uma variante de modelagem de linguagem, <font color="yellow">por exemplo:</font> `BERT` com modelagem de linguagem mascarada (Masked), `GPT-2` com modelagem de linguagem causal.

A modelagem de linguagem também pode ser útil fora do pré-treinamento, <font color="yellow">por exemplo,</font> para mudar a distribuição do modelo para um <font color="pink">domínio específico</font>: usando um modelo de linguagem treinado em um corpus muito grande e, em seguida, ajustando-o (`fine-tuning`) para um `Dataset de notícias` ou `artigos científicos`, por exemplo [LysandreJik/arxiv-nlp](https://huggingface.co/lysandre/arxiv-nlp).

## <font color="red">Masked Language Modeling (Modelagem de Linguagem Mascarada)</font>

A `modelagem de linguagem mascarada` é a tarefa de mascarar tokens em uma sequência com um token de mascaramento e solicitar que o modelo preencha essa máscara com um token apropriado. Isso permite que o modelo atenda tanto ao contexto direito (`tokens à direita da máscara`) quanto ao contexto esquerdo (`tokens à esquerda da máscara`). Esse treinamento cria uma base sólida para tarefas posteriores que exigem `contexto bidirecional`, como `SQuAD` (resposta a perguntas, consulte [Lewis, Lui, Goyal et al., parte 4.2](https://arxiv.org/abs/1910.13461)). Se desejar ajustar (`fine-tune`) um modelo em uma tarefa de **modelagem de linguagem mascarada**, você pode utilizar o script [run_mlm.py](https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py).

Aqui está um exemplo de uso de pipelines para substituir uma máscara (`mask`) de uma sequência:

In [28]:
from transformers import pipeline

unmasker = pipeline("fill-mask")

No model was supplied, defaulted to distilroberta-base and revision ec58a5b (https://huggingface.co/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.
Downloading:  98%|█████████▊| 1.31G/1.34G [2:10:37<02:37, 168kB/s]


Isso gera as sequências com a máscara preenchida (`Mask filled`), a pontuação de confiança e o `id` do token no vocabulário do tokenizer:

In [29]:
from pprint import pprint

pprint(
    unmasker(
        f"HuggingFace is creating a {unmasker.tokenizer.mask_token} that the community uses to solve NLP tasks."
    )
)

[{'score': 0.17927584052085876,
  'sequence': 'HuggingFace is creating a tool that the community uses to solve '
              'NLP tasks.',
  'token': 3944,
  'token_str': ' tool'},
 {'score': 0.11349426209926605,
  'sequence': 'HuggingFace is creating a framework that the community uses to '
              'solve NLP tasks.',
  'token': 7208,
  'token_str': ' framework'},
 {'score': 0.05243551358580589,
  'sequence': 'HuggingFace is creating a library that the community uses to '
              'solve NLP tasks.',
  'token': 5560,
  'token_str': ' library'},
 {'score': 0.03493541106581688,
  'sequence': 'HuggingFace is creating a database that the community uses to '
              'solve NLP tasks.',
  'token': 8503,
  'token_str': ' database'},
 {'score': 0.02860243059694767,
  'sequence': 'HuggingFace is creating a prototype that the community uses to '
              'solve NLP tasks.',
  'token': 17715,
  'token_str': ' prototype'}]


Aqui está um exemplo de `modelagem de linguagem mascarada` usando um modelo e um tokenizador. O processo é o seguinte:

<font color="yellow">1.</font> Instancie um `tokenizador` e um `modelo` a partir do nome do checkpoint. O modelo é identificado como um modelo `DistilBERT` e o carrega com os pesos armazenados no ponto de verificação.

<font color="yellow">2.</font> Defina uma sequência com um `token mascarado`, colocando o `tokenizer.mask_token` em vez de uma palavra.

<font color="yellow">3.</font> Codifique essa sequência em uma lista de IDs e encontre a posição do `token mascarado` nessa lista.

<font color="yellow">4.</font> Recupere as previsões no índice do token de máscara: ``esse tensor tem o mesmo tamanho do vocabulário`` e os valores são as pontuações atribuídas a cada token. O modelo atribui maior pontuação aos tokens que considera prováveis ​​naquele contexto.

<font color="yellow">5.</font> Recupere os 5 principais tokens usando os métodos `topk` do `PyTorch` ou `top_k` do TensorFlow.

<font color="yellow">6.</font> Substitua o token de máscara pelos tokens e imprima os resultados

In [32]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
model = AutoModelForMaskedLM.from_pretrained("distilbert-base-cased")

sequence = (
    "Distilled models are smaller than the models they mimic. Using them instead of the large "
    f"versions would help {tokenizer.mask_token} our carbon footprint."
)

inputs = tokenizer(sequence, return_tensors="pt")
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

token_logits = model(**inputs).logits
mask_token_logits = token_logits[0, mask_token_index, :]

top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))

Distilled models are smaller than the models they mimic. Using them instead of the large versions would help reduce our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help increase our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help decrease our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help offset our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help improve our carbon footprint.


In [34]:
from transformers import TFAutoModelForMaskedLM, AutoTokenizer
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
model = TFAutoModelForMaskedLM.from_pretrained("distilbert-base-cased")

sequence = (
    "Distilled models are smaller than the models they mimic. Using them instead of the large "
    f"versions would help {tokenizer.mask_token} our carbon footprint."
)

inputs = tokenizer(sequence, return_tensors="tf")
mask_token_index = tf.where(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]

token_logits = model(**inputs).logits
mask_token_logits = token_logits[0, mask_token_index, :]

top_5_tokens = tf.math.top_k(mask_token_logits, 5).indices.numpy()

for token in top_5_tokens:
    print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))

Some layers from the model checkpoint at distilbert-base-cased were not used when initializing TFDistilBertForMaskedLM: ['activation_13']
- This IS expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertForMaskedLM were initialized from the model checkpoint at distilbert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForMaskedLM for predictions without further training.


Distilled models are smaller than the models they mimic. Using them instead of the large versions would help reduce our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help increase our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help decrease our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help offset our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help improve our carbon footprint.


Isso imprime cinco sequências, com os $5$ principais tokens previstos pelo modelo.

## <font color="red">Causal Language Modeling (Modelagem de Linguagem Causal)</font>

A `modelagem de linguagem causal` é a tarefa de prever o token `seguindo uma sequência de tokens`. Nesta situação, o modelo atende apenas ao contexto esquerdo (`tokens à esquerda da máscara`). Tal treinamento é particularmente interessante para `tarefas de geração`. Se desejar ajustar (`fine-tune`) um modelo em uma tarefa de `modelagem de linguagem causal`, você pode aproveitar o script [run_clm.py](https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_clm.py).

Normalmente, o próximo token é previsto pela amostragem dos logits do último estado oculto que o modelo produz a partir da sequência de entrada.

Aqui está um exemplo de como usar o tokenizador e modelo e alavancar o método [top_k_top_p_filtering()](https://huggingface.co/docs/transformers/main/en/internal/generation_utils#transformers.top_k_top_p_filtering) para amostrar o próximo token após uma sequência de entrada de tokens.

In [37]:
from transformers import AutoModelForCausalLM, AutoTokenizer, top_k_top_p_filtering
import torch
from torch import nn


tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

sequence = f"Hugging Face is based in DUMBO, New York City, and"

inputs = tokenizer(sequence, return_tensors="pt")
input_ids = inputs["input_ids"]

# get logits of last hidden state
next_token_logits = model(**inputs).logits[:, -1, :]

# filter
filtered_next_token_logits = top_k_top_p_filtering(next_token_logits, top_k=50, top_p=1.0)

# sample
probs = nn.functional.softmax(filtered_next_token_logits, dim=-1)
next_token = torch.multinomial(probs, num_samples=1)

generated = torch.cat([input_ids, next_token], dim=-1)

resulting_string = tokenizer.decode(generated.tolist()[0])
print(resulting_string)

Hugging Face is based in DUMBO, New York City, and features


Aqui está um exemplo de como usar o `tokenizador` e `modelo` e alavancar o método [tf_top_k_top_p_filtering()](https://huggingface.co/docs/transformers/main/en/internal/generation_utils#transformers.tf_top_k_top_p_filtering) para amostrar o próximo token após uma sequência de entrada de tokens.

In [42]:
from transformers import TFAutoModelForCausalLM, AutoTokenizer, tf_top_k_top_p_filtering
import tensorflow as tf


tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = TFAutoModelForCausalLM.from_pretrained("gpt2")

sequence = f"Hugging Face is based in DUMBO, New York City, and"

inputs = tokenizer(sequence, return_tensors="tf")
input_ids = inputs["input_ids"]

# get logits of last hidden state
next_token_logits = model(**inputs).logits[:, -1, :]

# filter
filtered_next_token_logits = tf_top_k_top_p_filtering(next_token_logits, top_k=50, top_p=1.0)

# sample
next_token = tf.random.categorical(filtered_next_token_logits, dtype=tf.int32, num_samples=1)

generated = tf.concat([input_ids, next_token], axis=1)

resulting_string = tokenizer.decode(generated.numpy().tolist()[0])
print(resulting_string)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Hugging Face is based in DUMBO, New York City, and focuses


Isso gera um próximo token (espero) coerente seguindo a sequência original, que em nosso caso é a palavra `is` ou `features`.

Na próxima seção, mostramos como [generation.GenerationMixin.generate()](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationMixin.generate) pode ser usado para `gerar vários tokens até um comprimento especificado em vez de um token por vez`.

## <font color="red">Text Generation (Geração de texto)</font>

Na `geração de texto` (também conhecido `como geração de texto aberto`), o objetivo é criar uma parte coerente do texto que seja uma continuação do contexto dado. O exemplo a seguir mostra como `GPT-2` pode ser usado em pipelines para gerar texto. Como padrão, todos os modelos aplicam a amostragem `Top-K` quando usados ​​em pipelines, conforme configurado em suas respectivas configurações (consulte a [gpt-2 config](https://huggingface.co/gpt2/blob/main/config.json), por exemplo).

In [49]:
from transformers import pipeline

text_generator = pipeline("text-generation")
print(text_generator("As far as I am concerned, I will", max_length=50, do_sample=False))

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'As far as I am concerned, I will be the first to admit that I am not a fan of the idea of a "free market." I think that the idea of a free market is a bit of a stretch. I think that the idea'}]


Aqui, o modelo gera um texto aleatório com um `comprimento máximo total de $50$ tokens` do contexto "As far as I am concerned, I will". Nos bastidores, o objeto pipeline chama o método [PreTrainedModel.generate()](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationMixin.generate) para gerar texto. Os argumentos padrão para esse método podem ser substituídos no pipeline, conforme mostrado acima para os argumentos `max_length` e `do_sample`.

Segue abaixo um exemplo de `geração de texto` utilizando `XLNet` e `seu tokenizer`, que inclui diretamente a chamada `generate()`:

In [51]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("xlnet-base-cased")
tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")

# Padding text helps XLNet with short prompts - proposed by Aman Rusia in https://github.com/rusiaaman/XLNet-gen#methodology
PADDING_TEXT = """In 1991, the remains of Russian Tsar Nicholas II and his family
(except for Alexei and Maria) are discovered.
The voice of Nicholas's young son, Tsarevich Alexei Nikolaevich, narrates the
remainder of the story. 1883 Western Siberia,
a young Grigori Rasputin is asked by his father and a group of men to perform magic.
Rasputin has a vision and denounces one of the men as a horse thief. Although his
father initially slaps him for making such an accusation, Rasputin watches as the
man is chased outside and beaten. Twenty years later, Rasputin sees a vision of
the Virgin Mary, prompting him to become a priest. Rasputin quickly becomes famous,
with people, even a bishop, begging for his blessing. <eod> </s> <eos>"""

prompt = "Today the weather is really nice and I am planning on "
inputs = tokenizer(PADDING_TEXT + prompt, add_special_tokens=False, return_tensors="pt")["input_ids"]

prompt_length = len(tokenizer.decode(inputs[0]))
outputs = model.generate(inputs, max_length=250, do_sample=True, top_p=0.95, top_k=60)
generated = prompt + tokenizer.decode(outputs[0])[prompt_length + 1 :]

print(generated)

Today the weather is really nice and I am planning on posting about this experience here tomorrow. This is also in the beginning of the month of February, which is my first time back home in January. As the moon rises, my own "love" (as I called it to the beginning of the month of February in 2015) (and I think that it probably was in this first month of January in January


In [52]:
from transformers import TFAutoModelForCausalLM, AutoTokenizer

model = TFAutoModelForCausalLM.from_pretrained("xlnet-base-cased")
tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")

# Padding text helps XLNet with short prompts - proposed by Aman Rusia in https://github.com/rusiaaman/XLNet-gen#methodology
PADDING_TEXT = """In 1991, the remains of Russian Tsar Nicholas II and his family
(except for Alexei and Maria) are discovered.
The voice of Nicholas's young son, Tsarevich Alexei Nikolaevich, narrates the
remainder of the story. 1883 Western Siberia,
a young Grigori Rasputin is asked by his father and a group of men to perform magic.
Rasputin has a vision and denounces one of the men as a horse thief. Although his
father initially slaps him for making such an accusation, Rasputin watches as the
man is chased outside and beaten. Twenty years later, Rasputin sees a vision of
the Virgin Mary, prompting him to become a priest. Rasputin quickly becomes famous,
with people, even a bishop, begging for his blessing. <eod> </s> <eos>"""

prompt = "Today the weather is really nice and I am planning on "
inputs = tokenizer(PADDING_TEXT + prompt, add_special_tokens=False, return_tensors="tf")["input_ids"]

prompt_length = len(tokenizer.decode(inputs[0]))
outputs = model.generate(inputs, max_length=250, do_sample=True, top_p=0.95, top_k=60)
generated = prompt + tokenizer.decode(outputs[0])[prompt_length + 1 :]

print(generated)

Downloading: 100%|██████████| 565M/565M [01:08<00:00, 8.28MB/s]  
/home/eddygiusepe/24_Transformers_NLP/NLP_Transformers/venv_transformers/lib/python3.8/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFXLNetLMHeadModel.

All the layers of TFXLNetLMHeadModel were initialized from the model checkpoint at xlnet-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetLMHeadModel for predictions without further training.


Today the weather is really nice and I am planning on writing for the fourth week of the next four. I was thinking of a picture of me and my mother and grandmother, who we have had for over a week now. There is this picture of my mom and me with my family in the family room in our living room. In one of the pictures I see my family eating breakfast with my mother and grandmother and I.


Atualmente, a geração de texto é possível com `GPT-2`, `OpenAi-GPT`, `CTRL`, `XLNet`, `Transfo-XL` e `Reformer` no `PyTorch` e também para a maioria dos modelos no `Tensorflow`. Como pode ser visto no exemplo acima, `XLNet` e `Transfo-XL` geralmente precisam ser preenchidos para funcionar bem. `GPT-2` <font color="yellow">geralmente é uma boa escolha para geração de texto aberto porque foi treinado em milhões de páginas da web com um objetivo de modelagem de linguagem causal.</font>

Para obter mais informações sobre como aplicar diferentes estratégias de decodificação para geração de texto, consulte também nossa postagem no blog de geração de texto [aqui](https://huggingface.co/blog/how-to-generate).

# Named Entity Recognition

`Named Entity Recognition` (NER) é a tarefa de classificar tokens de acordo com uma classe, </font color="yellow">por exemplo</font>, identificando um token como uma pessoa (`person`), uma organização ou um local. Um exemplo de Dataset de reconhecimento de entidade nomeada é o Dataset `CoNLL-2003`, que é totalmente baseado nessa tarefa. Se você deseja ajustar (`fine-tune`) um modelo em uma `tarefa NER`, pode aproveitar o script [run_ner.py](https://github.com/huggingface/transformers/blob/main/examples/pytorch/token-classification/run_ner.py).


Aqui está um exemplo de uso de pipelines para fazer `reconhecimento de entidade nomeada`, especificamente, tentando identificar tokens como pertencentes a uma das $9$ classes:

* `O`, Fora de uma entidade nomeada

* `B-MIS`, Início de uma entidade diversa (miscellaneous) logo após outra entidade diversa

* `I-MIS`, entidade diversa

* `B-PER`, início do nome de uma pessoa logo após o nome de outra pessoa

* `I-PER`, nome da pessoa

* `B-ORG`, Início de uma organização logo após outra organização

* `I-ORG`, Organização

* `B-LOC`, Início de um local logo após outro local

* `I-LOC`, Localização

Ele aproveita um modelo ajustado no `CoNLL-2003`, ajustado por [@stefan-it](https://github.com/stefan-it) de [dbmdz](https://github.com/dbmdz).

In [ ]:
from transformers import pipeline

ner_pipe = pipeline("ner")

sequence = """Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO,
therefore very close to the Manhattan Bridge which is visible from the window."""

Isso gera uma lista de todas as palavras que foram identificadas como uma das entidades das $9$ classes definidas acima. Aqui estão os resultados esperados:

In [3]:
for entity in ner_pipe(sequence):
    print(entity)

{'entity': 'I-ORG', 'score': 0.99957865, 'index': 1, 'word': 'Hu', 'start': 0, 'end': 2}
{'entity': 'I-ORG', 'score': 0.9909764, 'index': 2, 'word': '##gging', 'start': 2, 'end': 7}
{'entity': 'I-ORG', 'score': 0.9982224, 'index': 3, 'word': 'Face', 'start': 8, 'end': 12}
{'entity': 'I-ORG', 'score': 0.9994879, 'index': 4, 'word': 'Inc', 'start': 13, 'end': 16}
{'entity': 'I-LOC', 'score': 0.9994344, 'index': 11, 'word': 'New', 'start': 40, 'end': 43}
{'entity': 'I-LOC', 'score': 0.99931955, 'index': 12, 'word': 'York', 'start': 44, 'end': 48}
{'entity': 'I-LOC', 'score': 0.9993794, 'index': 13, 'word': 'City', 'start': 49, 'end': 53}
{'entity': 'I-LOC', 'score': 0.98625827, 'index': 19, 'word': 'D', 'start': 79, 'end': 80}
{'entity': 'I-LOC', 'score': 0.95142686, 'index': 20, 'word': '##UM', 'start': 80, 'end': 82}
{'entity': 'I-LOC', 'score': 0.9336589, 'index': 21, 'word': '##BO', 'start': 82, 'end': 84}
{'entity': 'I-LOC', 'score': 0.9761654, 'index': 28, 'word': 'Manhattan', 'star

Observe como os tokens da sequência `"Hugging Face"` foram identificados como uma organização, e `"New York City"`, `"DUMBO"` e `"Manhattan Bridge"` foram identificados como locais.

Aqui está um exemplo de reconhecimento de entidade nomeada, usando um modelo e um tokenizador. O processo é o seguinte:


<font color="orange">1.</font> Instancie um tokenizador e um modelo a partir do nome do checkpoint. O modelo é identificado como um `modelo BERT` e o carrega com os pesos armazenados no ponto de verificação.

<font color="orange">2.</font> Defina uma sequência com entidades conhecidas, como `"Hugging Face"` como uma organização e `"New York City"` como um local.

<font color="orange">3.</font> Divida as palavras em tokens para que possam ser mapeadas para previsões. Usamos um pequeno hack, primeiro, `codificando` e `decodificando` completamente a sequência, para que fiquemos com uma string que contém os tokens especiais.

<font color="orange">4.</font> Codifique (Encode) essa sequência em IDs (`tokens especiais são adicionados automaticamente`).

<font color="orange">5.</font> Recupere as previsões passando a entrada para o modelo e obtendo a primeira saída. Isso resulta em uma distribuição nas $9$ classes possíveis para cada token. Tomamos o `argmax` para recuperar a classe mais provável para cada token.

<font color="orange">6.</font> Compacte cada token com sua previsão e imprima-o.

In [5]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = (
    "Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO, "
    "therefore very close to the Manhattan Bridge."
)

inputs = tokenizer(sequence, return_tensors="pt")
tokens = inputs.tokens()

outputs = model(**inputs).logits
predictions = torch.argmax(outputs, dim=2)

In [8]:
predictions

tensor([[0, 6, 6, 6, 6, 0, 0, 0, 0, 0, 0, 8, 8, 8, 0, 0, 0, 0, 0, 8, 8, 8, 0, 0,
         0, 0, 0, 0, 8, 8, 0, 0]])

In [9]:
from transformers import TFAutoModelForTokenClassification, AutoTokenizer
import tensorflow as tf

model = TFAutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = (
    "Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO, "
    "therefore very close to the Manhattan Bridge."
)

inputs = tokenizer(sequence, return_tensors="tf")
tokens = inputs.tokens()

outputs = model(**inputs)[0]
predictions = tf.argmax(outputs, axis=2)

Downloading: 100%|██████████| 1.33G/1.33G [11:29<00:00, 1.94MB/s]  
2022-11-25 02:29:51.764730: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-11-25 02:29:51.765373: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (eddygiusepe): /proc/driver/nvidia/version does not exist
2022-11-25 02:29:51.790797: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Some layers from the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing TFBertForTokenClassification: ['dropout_147']
- This IS expected if you are initiali

Isso gera uma lista de cada token mapeado para sua previsão correspondente. Diferentemente do pipeline, aqui cada token tem uma predição, pois não removemos a classe `"0"`, o que significa que nenhuma entidade específica foi encontrada naquele token.

No exemplo acima, `predictions` é um número inteiro que corresponde à classe predita. Podemos utilizar a propriedade `model.config.id2label` para recuperar o nome da classe correspondente ao número da classe, que está ilustrado abaixo:

In [10]:
for token, prediction in zip(tokens, predictions[0].numpy()):
    print((token, model.config.id2label[prediction]))

('[CLS]', 'O')
('Hu', 'I-ORG')
('##gging', 'I-ORG')
('Face', 'I-ORG')
('Inc', 'I-ORG')
('.', 'O')
('is', 'O')
('a', 'O')
('company', 'O')
('based', 'O')
('in', 'O')
('New', 'I-LOC')
('York', 'I-LOC')
('City', 'I-LOC')
('.', 'O')
('Its', 'O')
('headquarters', 'O')
('are', 'O')
('in', 'O')
('D', 'I-LOC')
('##UM', 'I-LOC')
('##BO', 'I-LOC')
(',', 'O')
('therefore', 'O')
('very', 'O')
('close', 'O')
('to', 'O')
('the', 'O')
('Manhattan', 'I-LOC')
('Bridge', 'I-LOC')
('.', 'O')
('[SEP]', 'O')


# Summarization (resumo)

A `sumarização` é a tarefa de resumir um documento ou um artigo em um texto mais curto. Se quiser ajustar um modelo em uma tarefa de resumo, você pode aproveitar o script [run_summarization.py](https://github.com/huggingface/transformers/blob/main/examples/pytorch/summarization/run_summarization.py).

Um exemplo de conjunto de dados de summarization é o conjunto de dados `CNN/Daily Mail`, que consiste em artigos de notícias longos e foi criado para a tarefa de summarization. Se você quiser ajustar um modelo em uma tarefa de resumo, várias abordagens são descritas neste [documento](https://github.com/huggingface/transformers/blob/main/examples/pytorch/summarization/README.md).

Aqui está um exemplo de uso dos pipelines para fazer o resumo (summarization). Ele aproveita um `modelo de Bart` que foi ajustado no conjunto de dados da `CNN/Daily Mail`.

In [3]:
from transformers import pipeline

summarizer = pipeline("summarization")

ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Como o pipeline de summarization depende do método `PreTrainedModel.generate()`, podemos substituir os argumentos padrão `PreTrainedModel.generate()` diretamente no pipeline para `max_length` e `min_length` conforme mostrado abaixo. Isso gera o seguinte resumo:

In [8]:
print(summarizer(ARTICLE, max_length=130, min_length=50, do_sample=False))

[{'summary_text': ' Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002 . At one time, she was married to eight men at once, prosecutors say .'}]


Aqui está um exemplo de `summarization` usando um modelo e um tokenizador. O processo é o seguinte:

<font color="orange">1.</font> Instancie um tokenizador e um modelo a partir do nome do checkpoint. A sumarização geralmente é feita usando um modelo de `codificador-decodificador`, como `Bart` ou `T5`.

<font color="orange">2.</font> Defina o artigo que deve ser resumido (summarized).

<font color="orange">3.</font> Adicione o prefixo específico `T5` `"resumir:"`.

<font color="orange">4.</font> Use o método `PreTrainedModel.generate()` para gerar o resumo.

Neste exemplo, usamos o modelo `T5 do Google`. Embora tenha sido pré-treinado apenas em um Dataset misto multitarefa (incluindo `CNN/Daily Mail`), ele produz resultados muito bons.

In [10]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

# T5 uses a max_length of 512 so we cut the article to 512 tokens.
inputs = tokenizer("summarize: " + ARTICLE, return_tensors="pt", max_length=512, truncation=True)
outputs = model.generate(
    inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

prosecutors say the marriages were part of an immigration scam. if convicted, barrientos faces two criminal counts of "offering a false instrument for filing in the first degree" she has been married 10 times, nine of them between 1999 and 2002.


In [12]:
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer

model = TFAutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

# T5 uses a max_length of 512 so we cut the article to 512 tokens.
inputs = tokenizer("summarize: " + ARTICLE, return_tensors="tf", max_length=512)
outputs = model.generate(
    inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


prosecutors say the marriages were part of an immigration scam. if convicted, barrientos faces two criminal counts of "offering a false instrument for filing in the first degree" she has been married 10 times, nine of them between 1999 and 2002.


# Translation (Tradução)

`Tradução` é a tarefa de traduzir um texto de um idioma para outro. Se você deseja ajustar um modelo em uma tarefa de tradução, pode aproveitar o script [run_translation.py](https://github.com/huggingface/transformers/blob/main/examples/pytorch/translation/run_translation.py).


Um exemplo de um Dataset de tradução é o conjunto de dados `WMT` de inglês para `alemão`, que tem sentenças em inglês como dados de entrada e as sentenças correspondentes em alemão como dados de destino. Se você deseja ajustar um modelo em uma tarefa de tradução, várias abordagens são descritas neste [documento](https://github.com/huggingface/transformers/blob/main/examples/pytorch/translation/README.md).

Aqui está um exemplo de uso dos pipelines para fazer a tradução. Ele aproveita um `modelo T5` que foi pré-treinado apenas em um conjunto de dados de mistura multitarefa (`incluindo WMT`), ainda assim, produzindo resultados de tradução impressionantes.

In [13]:
from transformers import pipeline

translator = pipeline("translation_en_to_de")
print(translator("Hugging Face is a technology company based in New York and Paris", max_length=40))

No model was supplied, defaulted to t5-base and revision 686f1db (https://huggingface.co/t5-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'translation_text': 'Hugging Face ist ein Technologieunternehmen mit Sitz in New York und Paris.'}]


Como o pipeline de tradução depende do método `PreTrainedModel.generate()`, podemos substituir os argumentos padrão `PreTrainedModel.generate()` diretamente no pipeline, conforme mostrado `max_length` acima.

Aqui está um exemplo de tradução usando um modelo e um tokenizer. O processo é o seguinte:


<font color="orange">1.</font> Instancie um tokenizador e um modelo a partir do nome do ponto de verificação. A sumarização geralmente é feita usando um modelo de `codificador-decodificador`, como `Bart` ou `T5`.

<font color="orange">2.</font> Defina o artigo que deve ser resumido.

<font color="orange">3.</font> Adicione o prefixo específico `T5` `"translate English to German: "`

<font color="orange">4.</font> Use o método `PreTrainedModel.generate()` para realizar a tradução.

In [14]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

inputs = tokenizer(
    "translate English to German: Hugging Face is a technology company based in New York and Paris",
    return_tensors="pt",
)

outputs = model.generate(inputs["input_ids"], max_length=40, num_beams=4, early_stopping=True)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Hugging Face ist ein Technologieunternehmen mit Sitz in New York und Paris.


In [15]:
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer

model = TFAutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

inputs = tokenizer(
    "translate English to German: Hugging Face is a technology company based in New York and Paris",
    return_tensors="tf",
)

outputs = model.generate(inputs["input_ids"], max_length=40, num_beams=4, early_stopping=True)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Hugging Face ist ein Technologieunternehmen mit Sitz in New York und Paris.


Obtemos a mesma tradução do exemplo do pipeline.

# Audio classification

A `classificação de áudio` atribui uma classe a um sinal de áudio. O conjunto de dados `Keyword Spotting` do benchmark [SUPERB](https://huggingface.co/datasets/superb) é um exemplo de conjunto de dados que pode ser usado para o ajuste fino da classificação de áudio. Este conjunto de dados contém dez classes de palavras-chave para classificação. Se você quiser ajustar um modelo para `classificação de áudio`, consulte o script [run_audio_classification.py](https://github.com/huggingface/transformers/blob/main/examples/pytorch/audio-classification/run_audio_classification.py) ou este [guia de instruções](https://huggingface.co/docs/transformers/main/en/tasks/audio_classification).


Os exemplos a seguir demonstram como usar um [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline) e um modelo e tokenizador para inferência de classificação de áudio:

In [21]:
from transformers import pipeline
from datasets import load_dataset
import torch

torch.manual_seed(42)
dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
dataset = dataset.sort("id")
audio_file = dataset[0]["audio"]["path"]

audio_classifier = pipeline(
    task="audio-classification", model="ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition"
)

predictions = audio_classifier(audio_file)
predictions = [{"score": round(pred["score"], 4), "label": pred["label"]} for pred in predictions]
predictions

Found cached dataset librispeech_asr_demo (/home/eddygiusepe/.cache/huggingface/datasets/hf-internal-testing___librispeech_asr_demo/clean/2.1.0/d3bc4c2bc2078fcde3ad0f0f635862e4c0fef78ba94c4a34c4c250a097af240b)
/home/eddygiusepe/24_Transformers_NLP/NLP_Transformers/venv_transformers/lib/python3.8/site-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition were not used when initializing Wav2Vec2ForSequenceClassification: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.output.weight', 'classifier.output.bias']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassificati

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[{'score': 0.1315, 'label': 'calm'},
 {'score': 0.1307, 'label': 'neutral'},
 {'score': 0.1274, 'label': 'sad'},
 {'score': 0.1261, 'label': 'fearful'},
 {'score': 0.1242, 'label': 'happy'}]

In [23]:
# Reproduzindo o áudio
from IPython.display import Audio
Audio(audio_file)


O processo geral para usar um extrator de modelo e feature para classificação de áudio é:

<font color="orange">1.</font> Instancie um extrator de feature e um modelo a partir do nome do checkpoint.

<font color="orange">2.</font> Processe o sinal de áudio a ser classificado com um extrator de feature.

<font color="orange">3.</font> Passe a entrada pelo modelo e pegue o `argmax` para recuperar a classe mais provável.

<font color="orange">4.</font> Converta o `ID` da classe em um nome de classe `id2label` para retornar um resultado interpretável.

In [24]:
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification
from datasets import load_dataset
import torch

dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
dataset = dataset.sort("id")
sampling_rate = dataset.features["audio"].sampling_rate

feature_extractor = AutoFeatureExtractor.from_pretrained("superb/wav2vec2-base-superb-ks")
model = AutoModelForAudioClassification.from_pretrained("superb/wav2vec2-base-superb-ks")

inputs = feature_extractor(dataset[0]["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_ids = torch.argmax(logits, dim=-1).item()
predicted_label = model.config.id2label[predicted_class_ids]
predicted_label

Found cached dataset librispeech_asr_demo (/home/eddygiusepe/.cache/huggingface/datasets/hf-internal-testing___librispeech_asr_demo/clean/2.1.0/d3bc4c2bc2078fcde3ad0f0f635862e4c0fef78ba94c4a34c4c250a097af240b)
Downloading: 100%|██████████| 215/215 [00:00<00:00, 102kB/s]
Downloading: 100%|██████████| 2.42k/2.42k [00:00<00:00, 1.05MB/s]
/home/eddygiusepe/24_Transformers_NLP/NLP_Transformers/venv_transformers/lib/python3.8/site-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Downloading: 100%|██████████| 378M/378M [00:44<00:00, 8.47MB/s] 


'_unknown_'

# Automatic speech recognition (Reconhecimento automático de fala)

O `reconhecimento automático de fala` transcreve um sinal de áudio para texto. O Dataset do [Common Voice](https://huggingface.co/datasets/common_voice) é um exemplo de conjunto de dados que pode ser usado para fine-tuning do `reconhecimento automático de fala`. Ele contém um arquivo de áudio de um speaker e a frase correspondente. Se você quiser ajustar um modelo para reconhecimento automático de fala, consulte os scripts [run_speech_recognition_ctc.py](https://github.com/huggingface/transformers/blob/main/examples/pytorch/speech-recognition/run_speech_recognition_ctc.py) ou [run_speech_recognition_seq2seq.py](https://github.com/huggingface/transformers/blob/main/examples/pytorch/speech-recognition/run_speech_recognition_seq2seq.py) ou este [guia de instruções](https://huggingface.co/docs/transformers/main/en/tasks/asr).


Os exemplos a seguir demonstram como usar um [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline) e um modelo e tokenizador para inferência automática de reconhecimento de fala:

In [26]:
from transformers import pipeline
from datasets import load_dataset

dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
dataset = dataset.sort("id")
audio_file = dataset[0]["audio"]["path"]

speech_recognizer = pipeline(task="automatic-speech-recognition", model="facebook/wav2vec2-base-960h")
speech_recognizer(audio_file)

Found cached dataset librispeech_asr_demo (/home/eddygiusepe/.cache/huggingface/datasets/hf-internal-testing___librispeech_asr_demo/clean/2.1.0/d3bc4c2bc2078fcde3ad0f0f635862e4c0fef78ba94c4a34c4c250a097af240b)
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


{'text': 'MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL'}

In [27]:
# Reproduzindo o áudio ASR
from IPython.display import Audio
Audio(audio_file)


O processo geral para usar um `modelo` e `processador` para reconhecimento automático de fala é:

<font color="orange">1.</font> Instancie um `processador` (que reagrupa um extrator de feature para processamento de entrada e um tokenizador para decodificação) e um `modelo` a partir do nome do ponto de verificação.

<font color="orange">2.</font> Processe o sinal de áudio e o texto com um processador.

<font color="orange">3.</font> Passe a entrada pelo modelo e pegue o `argmax` para recuperar o texto previsto.

<font color="orange">4.</font> Decodifique o texto com um tokenizador para obter a transcrição.

In [29]:
from transformers import AutoProcessor, AutoModelForCTC
from datasets import load_dataset
import torch

dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
dataset = dataset.sort("id")
sampling_rate = dataset.features["audio"].sampling_rate

processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base-960h")
model = AutoModelForCTC.from_pretrained("facebook/wav2vec2-base-960h")

inputs = processor(dataset[0]["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits
predicted_ids = torch.argmax(logits, dim=-1)

transcription = processor.batch_decode(predicted_ids)
transcription[0]

Found cached dataset librispeech_asr_demo (/home/eddygiusepe/.cache/huggingface/datasets/hf-internal-testing___librispeech_asr_demo/clean/2.1.0/d3bc4c2bc2078fcde3ad0f0f635862e4c0fef78ba94c4a34c4c250a097af240b)
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


'MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL'

# Image classification

Assim como a classificação de texto e áudio, a `classificação de imagem` atribui uma classe a uma imagem. O Dataset [CIFAR-100](https://huggingface.co/datasets/cifar100) é um exemplo de conjunto de dados que pode ser usado para `fine-tuning` de classificação de imagem. Ele contém uma imagem e a classe correspondente. Se você quiser ajustar um modelo para classificação de imagens, consulte o script [run_image_classification.py](https://github.com/huggingface/transformers/blob/main/examples/pytorch/image-classification/run_image_classification.py) ou este [guia de instruções](https://huggingface.co/docs/transformers/main/en/tasks/image_classification).

Os exemplos a seguir demonstram como usar um [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline) e um modelo e tokenizador para inferência de classificação de image:

In [31]:
from transformers import pipeline

vision_classifier = pipeline(task="image-classification")
result = vision_classifier(
    images="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg"
)
print("\n".join([f"Class {d['label']} with score {round(d['score'], 4)}" for d in result]))

No model was supplied, defaulted to google/vit-base-patch16-224 and revision 5dca96d (https://huggingface.co/google/vit-base-patch16-224).
Using a pipeline without specifying a model name and revision in production is not recommended.


Class lynx, catamount with score 0.4335
Class cougar, puma, catamount, mountain lion, painter, panther, Felis concolor with score 0.0348
Class snow leopard, ounce, Panthera uncia with score 0.0324
Class Egyptian cat with score 0.0239
Class tiger cat with score 0.0229


![](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg)

O processo geral para usar um modelo e extrator de feature para classificação de imagens é:

<font color="orange">1.</font> Instancie um extrator de feature e um modelo a partir do nome do ponto de verificação.

<font color="orange">2.</font> Processe a imagem a ser classificada com um extrator de feature.

<font color="orange">3.</font> Passe a entrada pelo modelo e pegue o `argmax` para recuperar a classe prevista.

<font color="orange">4.</font> Converta o `ID da classe` em um nome de classe `id2label` para retornar um resultado interpretável.

In [32]:
from transformers import AutoFeatureExtractor, AutoModelForImageClassification
import torch
from datasets import load_dataset

dataset = load_dataset("huggingface/cats-image")
image = dataset["test"]["image"][0]

feature_extractor = AutoFeatureExtractor.from_pretrained("google/vit-base-patch16-224")
model = AutoModelForImageClassification.from_pretrained("google/vit-base-patch16-224")

inputs = feature_extractor(image, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predicted_label = logits.argmax(-1).item()
print(model.config.id2label[predicted_label])

No config specified, defaulting to: cats-image/image


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 545.00it/s]


Dataset cats-image downloaded and prepared to /home/eddygiusepe/.cache/huggingface/datasets/huggingface___cats-image/image/1.9.0/68fbc793fb10cd165e490867f5d61fa366086ea40c73e549a020103dcb4f597e. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 197.99it/s]


Egyptian cat
